In [1]:
import subprocess
import os
import datetime
import ipywidgets as widgets
import pandas as pd
import numpy as np
import folium
from folium import plugins
import branca.colormap as cm
import matplotlib.pyplot as plt
import mplleaflet
#from matplotlib.patches import Ellipse
import math

In [17]:
lat = widgets.FloatText(
    value=33.0,
    description='Lat',
    step=0.1
)
lon = widgets.FloatText(
    value=-115.0,
    description='Lon',
    step=0.1
)
width = widgets.FloatText(
    value=1,
    description='Width (degree)',
    step=0.1
)
height = widgets.FloatText(
    value=1,
    description='Height (degree)',
    step=0.1
)
analysisCenter = widgets.RadioButtons( 
    options=['JPL', 'NGL'], description ='Data Source',disabled=False)

t1 = widgets.DatePicker(description='Time 1',value = datetime.date(2010,4,7))
t2 = widgets.DatePicker(description='Time 2',value = datetime.date(2010,4,9))
file_name = widgets.Text(description='Output name', value='displacement')
all_widgets = [lat, lon, width, height, t1, t2, file_name,analysisCenter]
widgets.GridBox(all_widgets, layout=widgets.Layout(
    grid_template_columns='repeat(2,500px)'))

GridBox(children=(FloatText(value=33.0, description='Lat', step=0.1), FloatText(value=-115.0, description='Lon…

In [23]:
#disp_args = f'python getDisplacement.py --lat {lat.value} --lon {lon.value} --width {width.value} --height {height.value} -t1 {t1.value.strftime("%Y-%m-%d")} -t2 {t2.value.strftime("%Y-%m-%d")} -o {file_name.value}'
# parameters dict object
# required
parameters = {"lat":lat.value,"lon":lon.value,"width":width.value,"height":height.value,"epoch1":t1.value.strftime("%Y-%m-%d"),"epoch2":t2.value.strftime("%Y-%m-%d"),"output":file_name.value,'analysisCenter':analysisCenter.value}
# optional
parameters['scale'] = None
parameters['ref'] = None 
parameters['ct'] = None 
parameters['pt'] = None
parameters['eon'] = True
parameters['mon'] = False
parameters['dwin1'] = None
parameters['dwin2'] = None
parameters['vabs'] = False

In [4]:
# # turn dict object to a class object
# class objdict(dict):

#     def __getattr__(self, name):
#         if name in self:
#             return self[name]
#         else:
#             raise AttributeError("No such attribute: " + name)

#     def __setattr__(self, name, value):
#         self[name] = value

#     def __delattr__(self, name):
#         if name in self:
#             del self[name]
#         else:
#             raise AttributeError("No such attribute: " + name)


In [5]:
#process = subprocess.getoutput(disp_args)
from getDisplacementNGL import getDisplacement

In [24]:
data_table = getDisplacement(parameters)

In [25]:
datadf = pd.DataFrame(data_table, columns=["Site","Lon","Lat","Delta E","Delta N","Delta V","Sigma E","Sigma N","Sigma V"])

In [26]:
datadf

,Site,Lon,Lat,Delta E,Delta N,Delta V,Sigma E,Sigma N,Sigma V
0,AZYU,-114.527108,32.658350,3.726861,0.049331,-0.305040,0.320236,0.366350,1.254914
1,GMPK,-114.827345,33.051085,1.618383,-1.354073,-0.257955,0.357105,0.408148,1.547169
2,IID2,-115.031803,32.706169,4.961177,-2.973279,-0.635058,0.337953,0.391099,1.404379
3,MEXI,-115.475703,32.632992,96.402631,-100.849042,2.331893,0.742206,0.851125,3.132293
4,P499,-115.487912,32.979607,1.699417,-6.196397,1.193078,0.330997,0.388696,1.395059
5,P500,-115.299932,32.690047,8.059367,-8.197374,0.479759,0.317879,0.377277,1.289625
6,P501,-115.397912,32.875757,2.136546,-7.219379,1.852062,0.327634,0.389797,1.351847
7,P502,-115.421911,32.982452,1.651604,-5.779813,0.934865,0.329442,0.394529,1.357949
8,P508,-115.428705,33.247781,0.499068,-3.417934,1.127580,0.329365,0.388038,1.348236
9,P509,-115.293917,32.890664,2.450181,-5.178598,1.165128,0.316323,0.375699,1.279560


In [9]:
def create_map(df, lon_col='Lon', lat_col='Lat', data_col='Delta V', site_col='Site'):
    '''
    Parameters
    ----------
    df :
        Pandas dataframe with column of lat, long, and a data column
    lon_col:
        Name of column with longitude coords. Default value of 'Lon'
    lat_col:
        Name of column with lattitude coords. Default value of 'Lat'
    data_col:
        Name of column with values to be colormapped. Default is 'Delta N'
    site_col:
        Name of column with GPS site names. 
    -------
    Returns:
        Folium map object
    '''
    start_cords = (df[lat_col].median(),
                   df[lon_col].median())
    my_map = folium.Map(start_cords,
                        tiles=None,zoom_start=8, width=750, height=465,contol_scale=True)
    plugins.Fullscreen(position='topleft').add_to(my_map)
    # load basemap with name
    folium.TileLayer(tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Topo_Map/MapServer/tile/{z}/{y}/{x}',
                        attr='Tiles &copy; Esri', name='World_Topo_Map').add_to(my_map)

    
    # making MPL seismic colormap in Branca
    colors_list = ['blue', 'white', 'red']
    minimum = df[data_col].min()
    maximum = df[data_col].max()
    colormap = cm.LinearColormap(
        colors=colors_list, vmin=minimum, vmax=maximum)
    stations = folium.FeatureGroup(
        name='Stations')
    vertical = folium.FeatureGroup(
        name='Vertical Displacement')
    horizontal = folium.FeatureGroup(
        name='Horizontal Displacement')
    # displace scale, mm/yer/deg
    scale = 320
    icon_url = "https://maps.google.com/mapfiles/kml/paddle/grn-blank.png"
    html = '<a target="_blank" href="https://sideshow.jpl.nasa.gov/post/links/{site}.html"><img src="https://sideshow.jpl.nasa.gov/post/plots/{site}.jpg" width="300" height="300"></a>'
    for cord, disp, site in zip(zip(df[lat_col], df[lon_col]), df[data_col], df[site_col]):
        folium.Circle(
            location=cord,
            radius=abs(disp)*scale,
            fill=True,
            color=colormap(disp),
            popup=f'Vertical Displacement:{"{:10.4f}".format(disp)} mm,   Site:{site}',
            stroke=True,
            weight=1,
            fill_opacity=.8
        ).add_to(vertical)
        icon = folium.features.CustomIcon(icon_url,icon_size=(16, 16),icon_anchor=(8,16))
        #iframe = folium.element.IFrame(html=html, width=500, height=300)
        s_html = html.format(site=site)
        popup = folium.Popup(s_html, max_width=2650)

        folium.Marker(
            location=cord,
            icon=icon,
            popup=popup,
        ).add_to(stations)
    
    horizontal = folium.FeatureGroup(
        name='Horizontal Displacement')
    for lat,lon,vlon,vlat in zip(df[lat_col], df[lon_col],df['Delta E'],df['Delta N']):
        linedis = [[lat,lon],[lat+vlat/scale,lon+vlon/scale/math.cos(lat*math.pi/180.)]]
        folium.PolyLine(
            locations=linedis,
            color='red',
            weight=2
        ).add_to(horizontal)
                   
    errorlayer = folium.FeatureGroup(
        name='Error eclipse')
    for lat,lon,vlon,vlat,slon,slat in zip(df[lat_col], df[lon_col],df['Delta E'],df['Delta N'],df['Sigma E'],df['Sigma N']):
        theta = 0
        coords = []
        for k in range(0,31):
            angle = k/30*2*math.pi
            elon = slon*math.cos(angle)*math.cos(theta)-slat*math.sin(angle)*math.sin(theta)
            elat = slon*math.cos(angle)*math.sin(theta)+slat*math.sin(angle)*math.cos(theta)
            elon = (elon+vlon)/scale/math.cos(lat*math.pi/180.)
            elat = (elat+vlat)/scale
            coords.append([lat+elat,lon+elon])
        folium.Polygon(
            locations=coords,
            fill=False,
            color="black",
            weight=2 
        ).add_to(errorlayer)
    my_map.add_child(stations)
    my_map.add_child(vertical)
    my_map.add_child(horizontal)
    my_map.add_child(errorlayer)
    my_map.add_child(colormap)
#     folium.raster_layers.ImageOverlay('error_elippse.png', [[df[lat_col].min(
#     ), df[lon_col].min()], [df[lat_col].max(), df[lon_col].max()]], name='Error Ellipse').add_to(my_map)
    my_map.add_child(folium.LayerControl(postion='bottomleft'))
    
    return my_map

In [27]:
# I have no idea why the arrows are plotting off center from the circles. Seems super inconsistent. 
create_map(datadf)